In [18]:
import pandas as pd
import json
import ast

In [2]:
# Read in individual score progressions from the file
df = pd.read_csv("2022_score_progressions.csv")
df

,date,home,away,score_progression
0,2022-04-29,LA,NC,"[(0, [0, 0]), (3, [1, 0]), (13, [2, 0]), (51, ..."
1,2022-04-30,POR,KC,"[(0, [0, 0]), (8, [1, 0]), (34, [2, 0]), (51, ..."
2,2022-04-30,CHI,LOU,"[(0, [0, 0]), (2, [1, 0]), (10, [1, 1]), (25, ..."
3,2022-05-01,WAS,RGN,"[(0, [0, 0]), (14, [1, 0]), (53, [1, 1]), (66,..."
4,2022-05-01,HOU,SD,"[(0, [0, 0]), (86, [0, 1])]"
...,...,...,...,...
132,2022-10-16,HOU,KC,"[(0, [0, 0]), (5, [0, 1]), (21, [1, 1]), (90, ..."
133,2022-10-16,SD,CHI,"[(0, [0, 0]), (10, [0, 1]), (67, [1, 1]), (90,..."
134,2022-10-23,POR,SD,"[(0, [0, 0]), (8, [0, 1]), (20, [1, 1]), (90, ..."
135,2022-10-23,RGN,KC,"[(0, [0, 0]), (4, [0, 1]), (63, [0, 2])]"


In [37]:
# Filter to the regular season
df = df[df["date"] < "2022-10-03"]

In [41]:
# Parse the score progressions
df["score_progression"] = list(map(ast.literal_eval, df["score_progression"]))

<ipython-input-41-c0d23473213d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["score_progression"] = list(map(ast.literal_eval, df["score_progression"]))


In [42]:
# Create a flipped version of the scores for the away team
def flip_scores(prog):
    flipped = []
    for (minute, scores) in prog:
        flipped.append((minute, [scores[1], scores[0]]))
    return flipped

df["away_score_progression"] = list(map(flip_scores, df["score_progression"]))
df

<ipython-input-42-1a4d15023906>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["away_score_progression"] = list(map(flip_scores, df["score_progression"]))


,date,home,away,score_progression,away_score_progression
0,2022-04-29,LA,NC,"[(0, [0, 0]), (3, [1, 0]), (13, [2, 0]), (51, ...","[(0, [0, 0]), (3, [0, 1]), (13, [0, 2]), (51, ..."
1,2022-04-30,POR,KC,"[(0, [0, 0]), (8, [1, 0]), (34, [2, 0]), (51, ...","[(0, [0, 0]), (8, [0, 1]), (34, [0, 2]), (51, ..."
2,2022-04-30,CHI,LOU,"[(0, [0, 0]), (2, [1, 0]), (10, [1, 1]), (25, ...","[(0, [0, 0]), (2, [0, 1]), (10, [1, 1]), (25, ..."
3,2022-05-01,WAS,RGN,"[(0, [0, 0]), (14, [1, 0]), (53, [1, 1]), (66,...","[(0, [0, 0]), (14, [0, 1]), (53, [1, 1]), (66,..."
4,2022-05-01,HOU,SD,"[(0, [0, 0]), (86, [0, 1])]","[(0, [0, 0]), (86, [1, 0])]"
...,...,...,...,...,...
127,2022-10-01,NJY,POR,"[(0, [0, 0]), (39, [1, 0]), (44, [1, 1]), (45,...","[(0, [0, 0]), (39, [0, 1]), (44, [1, 1]), (45,..."
128,2022-10-01,WAS,HOU,"[(0, [0, 0]), (14, [0, 1]), (36, [1, 1]), (58,...","[(0, [0, 0]), (14, [1, 0]), (36, [1, 1]), (58,..."
129,2022-10-01,LOU,KC,"[(0, [0, 0]), (90, [1, 0])]","[(0, [0, 0]), (90, [0, 1])]"
130,2022-10-01,RGN,ORL,"[(0, [0, 0]), (8, [1, 0]), (24, [2, 0]), (31, ...","[(0, [0, 0]), (8, [0, 1]), (24, [0, 2]), (31, ..."


In [43]:
def collect_scores(team):
    home_scores = df[df["home"] == team]
    home_scores = home_scores[["date", "score_progression"]]

    away_scores = df[df["away"] == team]
    away_scores = away_scores[["date", "away_score_progression"]]
    away_scores = away_scores.rename(columns={"away_score_progression": "score_progression"})
    
    return pd.concat([home_scores, away_scores]).sort_values("date")

In [44]:
collect_scores("RGN")

,date,score_progression
3,2022-05-01,"[(0, [0, 0]), (14, [0, 1]), (53, [1, 1]), (66,..."
8,2022-05-08,"[(0, [0, 0]), (23, [0, 1]), (33, [1, 1]), (37,..."
10,2022-05-13,"[(0, [0, 0])]"
22,2022-05-22,"[(0, [0, 0])]"
24,2022-05-25,"[(0, [0, 0]), (80, [1, 0])]"
27,2022-05-29,"[(0, [0, 0]), (75, [1, 0])]"
34,2022-06-04,"[(0, [0, 0]), (47, [0, 1])]"
44,2022-06-12,"[(0, [0, 0]), (8, [0, 1]), (38, [1, 1])]"
49,2022-06-18,"[(0, [0, 0]), (31, [1, 0])]"
54,2022-07-01,"[(0, [0, 0]), (52, [1, 0]), (74, [2, 0])]"


In [45]:
# Get the list of all teams
teams = list(df["home"].unique())
teams

['LA',
 'POR',
 'CHI',
 'WAS',
 'HOU',
 'ORL',
 'SD',
 'KC',
 'RGN',
 'LOU',
 'NC',
 'NJY']

In [46]:
# Iterate over all teams to collect the scores
results = {}
for team in teams:
    results[team] = list(collect_scores(team)["score_progression"])
results

{'LA': [[(0, [0, 0]), (3, [1, 0]), (13, [2, 0]), (51, [2, 1])],
  [(0, [0, 0]), (3, [0, 1])],
  [(0, [0, 0]), (42, [1, 0])],
  [(0, [0, 0]), (70, [1, 0])],
  [(0, [0, 0]), (57, [0, 1])],
  [(0, [0, 0]), (15, [0, 1]), (18, [0, 2]), (68, [0, 3])],
  [(0, [0, 0])],
  [(0, [0, 0]),
   (29, [0, 1]),
   (36, [1, 1]),
   (47, [2, 1]),
   (65, [2, 2]),
   (90, [3, 2])],
  [(0, [0, 0]), (31, [0, 1])],
  [(0, [0, 0]), (2, [1, 0]), (90, [1, 1])],
  [(0, [0, 0]), (9, [1, 0]), (59, [1, 1]), (81, [2, 1])],
  [(0, [0, 0]),
   (9, [1, 0]),
   (28, [2, 0]),
   (54, [2, 1]),
   (64, [2, 2]),
   (89, [2, 3])],
  [(0, [0, 0]), (40, [1, 0]), (64, [1, 1]), (72, [2, 1]), (90, [2, 2])],
  [(0, [0, 0]), (48, [1, 0])],
  [(0, [0, 0]), (78, [1, 0]), (82, [1, 1])],
  [(0, [0, 0]), (16, [1, 0]), (25, [2, 0]), (31, [3, 0]), (52, [3, 1])],
  [(0, [0, 0]), (43, [0, 1]), (49, [1, 1])],
  [(0, [0, 0]), (54, [0, 1])],
  [(0, [0, 0]), (30, [0, 1])],
  [(0, [0, 0]), (39, [1, 0]), (51, [2, 0]), (88, [2, 1])],
  [(0, [0, 0]

In [48]:
with open("2022_score_progressions.json", "w") as outfile:
    json.dump(results, outfile)